## Preprocessing for the .xlsx file

In [8]:
# import libraries
import pandas as pd
from pathlib import Path

### Load data from excel sheet

In [9]:
# reads the patient excel sheet and puts them into a pandas Dataframe
path = "/Users/LennartPhilipp/Desktop/Uni/Prowiss/Dateien/Metas_PatID_Full_SMI private Datei.xlsx"
try:
    patientsDataFrame = pd.read_excel(path)
except ImportError as e:
    print(e)

modifiedPatientsDataFrame = patientsDataFrame.copy()

### Remove unnecessary columns

In [10]:
# removes patients' names from dataset
#modifiedPatientsDataFrame = modifiedPatientsDataFrame.drop(columns=["name"])

# removes unused columns from dataset
modifiedPatientsDataFrame = modifiedPatientsDataFrame.iloc[:, :38] # only keeps the columns until column 38
modifiedPatientsDataFrame = modifiedPatientsDataFrame.drop(columns=["T.1",
                                                                    "primary_coded",
                                                                    "primary_coded_clean",
                                                                    "N",
                                                                    "M",
                                                                    "M_number_organ_systems",
                                                                    "TNM",
                                                                    "Survival ab ED gesamt",
                                                                    "Survival ab ED Hirnmet.",
                                                                    "Survival_from_ED_Met (months)",
                                                                    "Alter bei MRT (J./kalkuliert)",
                                                                    "Dauer Tumorleiden\n(kalkuliert; von ED bis ED Hirnmet.)",
                                                                    "Karnofsky (%)",
                                                                    "periphere Metastasen zum Zeitpunkt des MRTs ( Keine = 0, Einzelne = 1, Oligo =2, Multipel =3)",
                                                                    "im Verlauf neu aufgetretene Metastasen (0 = keine bekannt)",
                                                                    ])

# removes patients that did not fit the study requirements
modifiedPatientsDataFrame = modifiedPatientsDataFrame[modifiedPatientsDataFrame["in_study (0 = no, 1 = yes, 2 = tbd, 3 = remarkable)"] != 0]

# once all the MRI files are on the server change the following line to only inclue patients with MRI files: modifiedDataFrame["on_server"] != 0
modifiedPatientsDataFrame = modifiedPatientsDataFrame[modifiedPatientsDataFrame["on_server"] != 0]

# turn all preop_MRICE_date strings into datetime format, ignoring strings that can't be turned into dates
modifiedPatientsDataFrame["preop_MRICE_date"] = pd.to_datetime(modifiedPatientsDataFrame["preop_MRICE_date"], errors="ignore")

# turn all brithdate, death_date, date_first_diagnosis_primary and date_first_diagnosis_brain_met strings into datetime format, setting all strings that can't be turned into dates to NaT
for column in ["birthdate", "death_date", "date_first_diagnosis_primary", "date_first_diagnosis_brain_met"]:
    modifiedPatientsDataFrame[column] = pd.to_datetime(modifiedPatientsDataFrame[column], errors="coerce")

# turn all weights into numerics
modifiedPatientsDataFrame["Gewicht (kg)"] = pd.to_numeric(modifiedPatientsDataFrame["Gewicht (kg)"], errors="coerce")

/var/folders/by/b05bkcn50tb0fx4h3hhz0h800000gn/T/ipykernel_97312/2285622944.py:34: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  modifiedPatientsDataFrame[column] = pd.to_datetime(modifiedPatientsDataFrame[column], errors="coerce")
/var/folders/by/b05bkcn50tb0fx4h3hhz0h800000gn/T/ipykernel_97312/2285622944.py:34: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  modifiedPatientsDataFrame[column] = pd.to_datetime(modifiedPatientsDataFrame[column], errors="coerce")


### CUP patients

In [4]:
cupPatients = modifiedPatientsDataFrame[modifiedPatientsDataFrame["in_study (0 = no, 1 = yes, 2 = tbd, 3 = remarkable)"] == 3]

### Check amount of missing values

In [9]:
# checks the amount of missing values for each column
for column in modifiedPatientsDataFrame.columns:
    amount_of_nan = modifiedPatientsDataFrame[column].isna().sum()
    print("# of missing values in " + column + ": " + str(amount_of_nan))

# of missing values in on_server: 0
# of missing values in in_study (0 = no, 1 = yes, 2 = tbd, 3 = remarkable): 0
# of missing values in reason: 85
# of missing values in MRI_comments: 340
# of missing values in ID: 0
# of missing values in birthdate: 0
# of missing values in sex (m/w): 0
# of missing values in preop_MRICE_date: 1
# of missing values in primary_full: 1
# of missing values in primary_coded_newv: 0
# of missing values in T: 1
# of missing values in death_date: 273
# of missing values in date_first_diagnosis_primary: 35
# of missing values in date_first_diagnosis_brain_met: 37
# of missing values in OP: 3
# of missing values in Chemotherapie: 3
# of missing values in Radiatio: 3
# of missing values in Immuntherapie: 2
# of missing values in Größe (cm): 98
# of missing values in Gewicht (kg): 90
# of missing values in ECOG: 106
# of missing values in AZ Beschreibung: 349
# of missing values in Resektionsstatus (0 = Komplettresektion; 1 = Rest/weitere Hirnfiliae, 2 = keine 

### Update ECOG values
We assume that patients that were in good shape ("guter AZ") also have an ECOG value of 0, even though this was not explicitally labeled as such.

In [13]:
#counter = 0
for index, patient in modifiedPatientsDataFrame.iterrows():
    if pd.isna(patient["ECOG"]) and patient["AZ Beschreibung"] == "guter AZ":
        #counter += 1
        #print(counter)
        patient["ECOG"] = 0


### Primaries Value Counts

In [10]:
modifiedPatientsDataFrame["primary_coded_newv"].value_counts()

primary_coded_newv
1c     96
12     64
1a     49
1e     32
2      26
1b     15
1g     13
23     12
5       9
29      7
1f      5
11g     5
1d      4
6       4
19      3
3       3
18      3
4       2
11b     2
11c     2
10      2
20      2
9       2
15      2
24      1
26a     1
11f     1
26b     1
14      1
27      1
8       1
25      1
13      1
21      1
22      1
16      1
11a     1
17      1
Name: count, dtype: int64

### Adds a new column for the age when the images were taken

In [1]:
# modifiedPatientsDataFrame["age_at_MRI"] = pd.to_datetime(modifiedPatientsDataFrame["preop_MRICE_date"]) - pd.to_datetime((modifiedPatientsDataFrame["birthdate"]))

### Export only patientID, birthdate and sex as .csv
Get all the patients where the following list of conditions is fulfilled:
- on_server = 1
- in_study != (0 OR 3)

In [11]:
exportVersion = modifiedPatientsDataFrame.copy()
exportVersion = exportVersion[["on_server", "in_study (0 = no, 1 = yes, 2 = tbd, 3 = remarkable)", "ID", "sex (m/w)", "birthdate", "primary_coded_newv"]]
exportVersion = exportVersion[exportVersion["in_study (0 = no, 1 = yes, 2 = tbd, 3 = remarkable)"] != (3 or 0)]

In [12]:
exportVersion.to_csv("/Users/LennartPhilipp/Desktop/Uni/Prowiss/Code/patientsIDsexbirthdateprimary.csv")

### Compare list of patients with MRIs

#### Load csv files

In [15]:
MRI_patientIDs = pd.read_csv("/Users/LennartPhilipp/Desktop/Uni/Prowiss/Code/MRI_patientIDs.csv")
patientList = pd.read_csv("/Users/LennartPhilipp/Desktop/Uni/Prowiss/Code/patients.csv")

In [33]:
#print("MRI patient IDs: ", MRI_patientIDs)
#print("Patient List: ", patientList["ID"])

# creates a new Dataframe for the patients that meat the following criteria:  all the necessary MRI sequences exist AND patient wasn't excluded from the study
patients_in_study_with_MRI = pd.DataFrame({"patientIDs":[]})

# iterates through all the IDs of the patients with the necessary MRI sequences and compares them with the list of patients in the study
for index, row in MRI_patientIDs.iterrows():

    # if patientID of the MRI sequences is also in the list of patientIDS that are part of the study, then add the ID to patients_in_study_with_MRI
    if row["patientIDs"] in list(map(int, patientList["ID"])):

        new_row = {"patientIDs":row["patientIDs"]}
        patients_in_study_with_MRI = pd.concat([patients_in_study_with_MRI, pd.DataFrame([new_row])], ignore_index=True)
    
print(patients_in_study_with_MRI)

    patientIDs
0    1199093.0
1    1220269.0
2    1150136.0
3    1134825.0
4    1083248.0
5    1189050.0
6    1311383.0
7    1261127.0
8    1243841.0
9    1152379.0
10   1106844.0
11   1241505.0
12   1307298.0
13   1269967.0
14   1190738.0
15   1111974.0
16   1038520.0
17   1321873.0
18   1122863.0
19   1108350.0
20   1009590.0
21   1022787.0
22   1055292.0
23   1314225.0
24   1288896.0
25   1005097.0
26   1109318.0


## Find new patient MRIs
The goal is to maybe find some more patient MRI scans that I can use for the dataset.\
First: get a list of all the patientIDs of the patients that I already have\
Second: get the dataframe of all the patients with in_study: 1 or 2\
Third: Compare the list and get all the patients that are in the excel sheet but already exported\
Four: Go through the list manually

In [46]:
# First step below

path_to_participants_tsv = Path("/Volumes/BrainMets/Rgb_Brain_Mets/brain_mets_classification/rawdata/participants.tsv")

path_to_sheets_dir = Path("/Users/LennartPhilipp/Desktop/Uni/Prowiss/Dateien")

participants = pd.read_csv(path_to_participants_tsv, sep="\t")
participants.drop(columns=["sex (m/f)", "age", "primary"])

def get_pat_id(participant_id):
    id = participant_id.split("-")[1]
    return int(id)

patient_ids = list(participants["participant_id"].apply(get_pat_id))

# Second stop below

modifiedPatientsDataFrame = modifiedPatientsDataFrame[modifiedPatientsDataFrame["in_study (0 = no, 1 = yes, 2 = tbd, 3 = remarkable)"] != 3]
modifiedPatientsDataFrame = modifiedPatientsDataFrame[modifiedPatientsDataFrame["in_study (0 = no, 1 = yes, 2 = tbd, 3 = remarkable)"] != 0]
modifiedPatientsDataFrame = modifiedPatientsDataFrame.iloc[:, :38] # only keeps the columns until column 38
modifiedPatientsDataFrame = modifiedPatientsDataFrame.drop(columns=["T.1",
                                                                    "primary_coded",
                                                                    "primary_coded_clean",
                                                                    "N",
                                                                    "M",
                                                                    "M_number_organ_systems",
                                                                    "TNM",
                                                                    "Survival ab ED gesamt",
                                                                    "Survival ab ED Hirnmet.",
                                                                    "Survival_from_ED_Met (months)",
                                                                    "Alter bei MRT (J./kalkuliert)",
                                                                    "Dauer Tumorleiden\n(kalkuliert; von ED bis ED Hirnmet.)",
                                                                    "Karnofsky (%)",
                                                                    "periphere Metastasen zum Zeitpunkt des MRTs ( Keine = 0, Einzelne = 1, Oligo =2, Multipel =3)",
                                                                    "im Verlauf neu aufgetretene Metastasen (0 = keine bekannt)",
                                                                    ])

# turn all preop_MRICE_date strings into datetime format, ignoring strings that can't be turned into dates
modifiedPatientsDataFrame["preop_MRICE_date"] = pd.to_datetime(modifiedPatientsDataFrame["preop_MRICE_date"], errors="ignore")

# turn all brithdate, death_date, date_first_diagnosis_primary and date_first_diagnosis_brain_met strings into datetime format, setting all strings that can't be turned into dates to NaT
for column in ["birthdate", "death_date", "date_first_diagnosis_primary", "date_first_diagnosis_brain_met"]:
    modifiedPatientsDataFrame[column] = pd.to_datetime(modifiedPatientsDataFrame[column], errors="coerce")


# Third step below

patients_without_images = modifiedPatientsDataFrame[~modifiedPatientsDataFrame["ID"].isin(patient_ids)]

patients_without_images.to_excel(path_to_sheets_dir / Path("patients_without_images.xlsx"), index=False)

/var/folders/by/b05bkcn50tb0fx4h3hhz0h800000gn/T/ipykernel_54471/1835813272.py:43: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  modifiedPatientsDataFrame[column] = pd.to_datetime(modifiedPatientsDataFrame[column], errors="coerce")
/var/folders/by/b05bkcn50tb0fx4h3hhz0h800000gn/T/ipykernel_54471/1835813272.py:43: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  modifiedPatientsDataFrame[column] = pd.to_datetime(modifiedPatientsDataFrame[column], errors="coerce")
